In [32]:
import logging
import re
import ssl
import time
from datetime import date
from functools import reduce

import numpy as np
import pandas as pd
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from slack_sdk.web.slack_response import SlackResponse
from sqlalchemy import MetaData, Table
from sqlalchemy.sql import and_, case, func, or_, select, text

from pax_achievements import (
    cadre,
    el_presidente,
    hammer_not_nail,
    hdtf,
    leader_of_men,
    nation_sql,
    posts,
    region_sql,
    six_pack,
    the_boss,
    the_monk,
    the_priest,
)
from utils import mysql_connection, slack_client


In [2]:
logging.basicConfig(format="%(asctime)s [%(levelname)s]:%(message)s",
                        level=logging.INFO,
                        datefmt="%Y-%m-%d %H:%M:%S")

In [3]:
year = date.today().year
engine = mysql_connection()
metadata = MetaData()
%time metadata.reflect(engine, schema="weaselbot")

CPU times: user 73.5 ms, sys: 8 ms, total: 81.5 ms
Wall time: 2.97 s


In [4]:
df_regions = pd.read_sql(region_sql(metadata), engine)
nation_df = pd.read_sql(nation_sql(metadata, year), engine, parse_dates="date")

In [5]:
bb_filter = nation_df.backblast.str.lower().str[:100].str.contains(r"q.{0,1}source|q{0,1}[1-9]\.[0-9]\s")
ao_filter = nation_df.ao.str.contains(r"q.{0,1}source")

In [29]:
dfs = list()
############# Q Source ##############
priest_df = the_priest(nation_df, bb_filter, ao_filter)
monk_df = the_monk(nation_df, bb_filter, ao_filter)
############### END #################

In [30]:
############ ALL ELSE ###############
leader_of_men_df = leader_of_men(nation_df, bb_filter, ao_filter)
boss_df = the_boss(nation_df, bb_filter, ao_filter)
hammer_not_nail_df = hammer_not_nail(nation_df, bb_filter, ao_filter)
cadre_df = cadre(nation_df, bb_filter, ao_filter)
el_presidente_df = el_presidente(nation_df, bb_filter, ao_filter)

ss = []
s = posts(nation_df, bb_filter, ao_filter)
for val in [25, 50, 100, 150, 200]:
    ss.append(s[s >= val])
el_quatro, golden_boy, centurian, karate_kid, crazy_person = ss
six_pack_df = six_pack(nation_df, bb_filter, ao_filter)
hdtf_df = hdtf(nation_df, bb_filter, ao_filter)

In [74]:
df_regions.paxminer_schema.unique()

array(['f3alliance', 'f3waco', 'f3naperville', 'f3waxhaw', 'f3columbia',
       'f3muletown', 'f3pugetsound', 'f3jeffco', 'f3stlmeramec',
       'f3southcary', 'f3glacier', 'f3ttown', 'f3washmo', 'f3expedition',
       'f3central_il', 'f3southsound', 'f3thecapital', 'f3nwarkansas',
       'f3thespark', 'f3gwinnett', 'f3stl', 'f3orlando', 'f3westcobb',
       'f3marshall', 'f3roanoke', 'f3albany', 'f3suncoast', 'f3austin',
       'f3hillcountry', 'f3blueridge', 'f3portland', 'f3northlake',
       'f3cherokee', 'f3denver', 'f3florence', 'f3fortcollins',
       'f3thejunction', 'f3wichita', 'f3noho', 'f3knoxville', 'f3thefort',
       'f3houston', 'f3kcmo', 'f3borderlands', 'f3westindy', 'f3soco',
       'f3huntsville', 'f3flomo', 'f3albuquerque', 'f3spiritofstl',
       'f3ne_wake', 'f3stcharles', 'f3thechuck', 'f3chicago',
       'f3se_michigan', 'f3brentwood', 'f3sclt', 'f3twincities',
       'f3memphis', 'f3noco', 'f3tornadoalley'], dtype=object)

In [75]:
for r in df_regions.itertuples(index=False):
    if r.paxminer_schema == "f3ne_wake":
        row = r

In [ ]:
# client = slack_client(row.slack_token) # no need for this yet. Here for completeness

In [77]:
idx = pd.IndexSlice
try:
    records = priest_df[idx[:, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=1, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in the_priest")

2024-01-28 19:15:15 [ERROR]:f3ne_wake not in the_priest


In [78]:
try:
    records = monk_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=2, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in the_monk")

In [79]:
data

[{'month': 1,
  'slack_user_id': 'U0328DW47MH',
  'achievement_id': 2,
  'date_awarded': datetime.date(2024, 1, 28)},
 {'month': 1,
  'slack_user_id': 'U032Y5T6VH6',
  'achievement_id': 2,
  'date_awarded': datetime.date(2024, 1, 28)}]

In [82]:
try:
    records = leader_of_men_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=3, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in leader_of_men")

In [83]:
data

[{'month': 1,
  'slack_user_id': 'U01QC9PN2LV',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 28)},
 {'month': 1,
  'slack_user_id': 'U0328DW47MH',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 28)},
 {'month': 1,
  'slack_user_id': 'U04HW3RLQ6Q',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 28)},
 {'month': 1,
  'slack_user_id': 'U04R1S9RGQ5',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 28)}]

In [84]:
try:
    records = boss_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=4, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in the_boss")

2024-01-28 19:21:38 [ERROR]:f3ne_wake not in the_boss


In [85]:
try:
    records = hammer_not_nail_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=5, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in be_the_hammer_not_the_nail")

2024-01-28 19:21:39 [ERROR]:f3ne_wake not in be_the_hammer_not_the_nail


In [86]:
try:
    records = cadre_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=6, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in cadre")

2024-01-28 19:21:42 [ERROR]:f3ne_wake not in cadre


In [87]:
try:
    records = el_presidente_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=7, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in el_presidente")

2024-01-28 19:21:42 [ERROR]:f3ne_wake not in el_presidente


In [88]:
try:
    records = el_quatro[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=8, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in el_quatro")

2024-01-28 19:21:44 [ERROR]:f3ne_wake not in el_quatro


In [89]:
try:
    records = golden_boy[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=9, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in golden_boy")

2024-01-28 19:21:45 [ERROR]:f3ne_wake not in golden_boy


In [90]:
try:
    records = centurian[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=10, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in centurian")

2024-01-28 19:21:45 [ERROR]:f3ne_wake not in centurian


In [91]:
try:
    records = karate_kid[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=11, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in karate_kid")

2024-01-28 19:21:47 [ERROR]:f3ne_wake not in karate_kid


In [92]:
try:
    records = crazy_person[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=12, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in crazy_person")

2024-01-28 19:21:50 [ERROR]:f3ne_wake not in crazy_person


In [93]:
try:
    records = six_pack_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=13, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in six_pack")

2024-01-28 19:21:50 [ERROR]:f3ne_wake not in six_pack


In [69]:
try:
    records = hdtf_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao", axis=1).assign(achievement_id=14, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in holding_down_the_fort")

2024-01-28 19:08:12 [ERROR]:f3chicago not in holding_down_the_fort


In [100]:
# under current logic, this man received 6-pack. But he hasn't earned it because 2 of his beatdowns were Q Source. New
# logic catches it.

nation_df.query("slack_user_id == 'U0328DW47MH' and '2024-01-21' <= date <= '2024-01-27' and ao != 'ao_q_source'")

,email,user_name,slack_user_id,ao_id,ao,date,year_num,month_num,week_num,day_num,q_flag,backblast,home_region
32463,ray.perry17@gmail.com,Green Thumb (Ray Perry),U0328DW47MH,4974,ao_agoge,2024-01-22,2024,1,3,22,0,"Backblast! 5's, 7's, 9's, & 11's\nDATE: 2024-0...",f3ne_wake
32475,ray.perry17@gmail.com,Green Thumb (Ray Perry),U0328DW47MH,4965,ao_corral,2024-01-23,2024,1,3,23,0,Backblast! Jan 23 Corral Workout\nDATE: 2024-0...,f3ne_wake
32476,ray.perry17@gmail.com,Green Thumb (Ray Perry),U0328DW47MH,4964,ao_max_bandwidth,2024-01-25,2024,1,3,25,0,Backblast! Max Bandwidth - Yoda\nDATE: 2024-01...,f3ne_wake
32477,ray.perry17@gmail.com,Green Thumb (Ray Perry),U0328DW47MH,4972,ao_red_dawn,2024-01-26,2024,1,3,26,1,Backblast! Hills? What Hills?\nDATE: 2024-01-2...,f3ne_wake
32479,ray.perry17@gmail.com,Green Thumb (Ray Perry),U0328DW47MH,4959,ao_farm,2024-01-27,2024,1,3,27,0,Backblast! 28th Birthday Bash\nDATE: 2024-01-2...,f3ne_wake
